# Clustering Crypto

In [24]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [3]:
# Load the crypto_data.csv dataset.
file_path = "resources/crypto_data.csv"
df_crypto = pd.read_csv(file_path, index_col=0)
df_crypto.head(10)
print(f'There are {len(df_crypto)} rows in the dataset before filtering.')

There are 1252 rows in the dataset before filtering.


In [4]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
filter = df_crypto["IsTrading"] == True
df_crypto_cleaned = df_crypto.loc[filter]

print(f'There are {len(df_crypto_cleaned)} rows in the dataset after filtering')
df_crypto_cleaned.head(10)

There are 1144 rows in the dataset after filtering


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [5]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
df_crypto_scoured = df_crypto_cleaned.drop(["IsTrading"], axis=1)
df_crypto_scoured.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [6]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE
for column in df_crypto_scoured.columns:
    print(f"Column {column} has {df_crypto_scoured[column].isnull().sum()} null values")

df_crypto_scoured = df_crypto_scoured.dropna()
df_crypto_scoured.shape

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


(685, 5)

In [7]:
for column in df_crypto_scoured.columns:
    print(f"Column {column} has {df_crypto_scoured[column].isnull().sum()} null values")


Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


In [8]:
# Keep the rows where coins are mined.
# YOUR CODE HERE
filter = df_crypto_scoured["TotalCoinsMined"] > 0
df_crypto_scoured = df_crypto_scoured.loc[filter]
df_crypto_scoured.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [9]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
df_coins = pd.DataFrame(df_crypto_scoured["CoinName"])
df_coins.head(5)

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [10]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
df_crypto_prepped = df_crypto_scoured.drop(["CoinName"], axis=1)
df_crypto_prepped.shape
df_crypto_prepped.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [11]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
df_crypto_dummies = pd.get_dummies(df_crypto_prepped, columns=["Algorithm", "ProofType"])
df_crypto_dummies.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Standardize the data with StandardScaler().
# YOUR CODE HERE
crypto_scaled = StandardScaler().fit_transform(df_crypto_dummies)
crypto_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [13]:
# Using PCA to reduce dimension to three principal components.
# YOUR CODE HERE
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.32596125,  1.03407652, -0.42721315],
       [-0.30927778,  1.03422352, -0.42729353],
       [ 2.29235183,  1.64258546, -0.49100619],
       ...,
       [ 0.32787182, -2.34069194,  0.28554617],
       [-0.18240932, -1.99621177,  0.29921888],
       [-0.27903755,  0.81847622, -0.16178451]])

In [14]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
df_crypto_pca = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=df_crypto_dummies.index
)
print(df_crypto_pca.shape)
df_crypto_pca.head(10)

(532, 3)


,PC 1,PC 2,PC 3
42,-0.325961,1.034077,-0.427213
404,-0.309278,1.034224,-0.427294
1337,2.292352,1.642585,-0.491006
BTC,-0.151691,-1.327191,0.132828
ETH,-0.164666,-2.054269,0.274932
LTC,-0.155958,-1.125300,0.020000
DASH,-0.401635,1.245745,-0.455541
XMR,-0.151018,-2.224858,0.308044
ETC,-0.163105,-2.054373,0.274938
ZEC,-0.182408,-1.996212,0.299219


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [15]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE
inertia = []
k = list(range(1, 11))

# Identify cluster count
for i in k:
    km = KMeans(n_clusters=i, random_state=1)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Define DF to plot the curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [16]:
# Initialize the K-Means model.
# YOUR CODE HERE
model = KMeans(n_clusters=4, random_state=42) # I found your easter egg, Eric
# Fit the model
# YOUR CODE HERE
model.fit(df_crypto_pca)
# Predict clusters
# YOUR CODE HERE
predictions = model.predict(df_crypto_pca)
predictions

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1,

In [17]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pca_df DataFrames on the same columns.
# YOUR CODE HERE
clustered_df = pd.concat([df_crypto_prepped, df_crypto_pca], axis="columns")
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
# YOUR CODE HERE
clustered_df = pd.concat([clustered_df, df_coins], axis="columns")
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
# YOUR CODE HERE
clustered_df["Class"] = model.labels_
# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.325961,1.034077,-0.427213,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.309278,1.034224,-0.427294,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.292352,1.642585,-0.491006,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.151691,-1.327191,0.132828,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.164666,-2.054269,0.274932,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.155958,-1.125300,0.020000,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.401635,1.245745,-0.455541,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.151018,-2.224858,0.308044,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.163105,-2.054373,0.274938,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.182408,-1.996212,0.299219,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [18]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="Class",
    symbol="Class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [19]:
# Create a table with tradable cryptocurrencies.
# YOUR CODE HERE
table = clustered_df.hvplot.table(columns=['CoinName','Algorithm','ProofType','TotalCoinSupply','TotalCoinsMined','Class'])
table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [20]:
# Print the total number of tradable cryptocurrencies.
# YOUR CODE HERE
print(f'There are {len(clustered_df)} tradable cryptocurrencies.')

There are 532 tradable cryptocurrencies.


In [21]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
# YOUR CODE HERE
scaler = MinMaxScaler()
coins_scaled = pd.DataFrame(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])
coins_scaled = scaler.fit_transform(coins_scaled)
coins_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [22]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE
plot_df = pd.DataFrame(coins_scaled, columns=[["TotalCoinSupply", "TotalCoinsMined"]], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE
plot_df["CoinName"] = clustered_df["CoinName"]
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE
plot_df["Class"] = clustered_df["Class"]
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,1
404,5.320000e-04,0.001066,404Coin,1
1337,3.141593e-01,0.029576,EliteCoin,1
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,1
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [23]:
plot_df

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000e+00,42 Coin,1
404,5.320000e-04,1.065855e-03,404Coin,1
1337,3.141593e-01,2.957551e-02,EliteCoin,1
BTC,2.100000e-05,1.810842e-05,Bitcoin,0
ETH,0.000000e+00,1.087731e-04,Ethereum,0
...,...,...,...,...
ZEPH,2.000000e-03,2.020225e-03,ZEPHYR,1
GAP,2.500000e-04,1.508199e-05,Gapcoin,1
BDX,1.400223e-03,9.901351e-04,Beldex,0
ZEN,2.100000e-05,7.370282e-06,Horizen,0


In [27]:
fig = px.scatter(
    plot_df,
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    color=["Class"]
)

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [26]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.hvplot.scatter(
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    hover_cols=["CoinName"],
    by="Class")

ValueError: Dimensions specified as a tuple must be a tuple consisting of the name and label not: ('TotalCoinSupply',)